In [ ]:
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, DensityMatrix, Pauli, Operator, random_statevector, partial_trace
from qiskit_aer import AerSimulator, Aer
import numpy as np
from scipy.linalg import sqrtm
import cvxpy as cp
import itertools
from IPython.display import clear_output
from winsound import Beep

def tensor_prod(*tensors):
    if len(tensors) == 2:
        return np.kron(tensors[0], tensors[1])
    else:
        return np.kron(tensors[0], tensor_prod(*tensors[1:]))
    
def hermitian(matrix):
    return np.allclose(matrix, matrix.conj().T)

def trace_one(matrix):
    return np.isclose(np.trace(matrix), 1)

def positive_semi_definite(matrix, tol=1e-8):
    return np.all(np.linalg.eigvals(matrix) + tol >= 0)

def is_legal(matrix):
    return hermitian(matrix) and trace_one(matrix) and positive_semi_definite(matrix)

def check_legal(matrix, print_errors=True):
    errors, legal = [], True
    if not hermitian(matrix):
        errors.append('not hermitian')
    if not trace_one(matrix):
        errors.append('trace not equal to one')
    if not positive_semi_definite(matrix):
        errors.append('not positive semidefinite')
    if len(errors) > 0:
        legal = False
    if print_errors:
        if not legal:
            print(f'input is not legal: ' + '; '.join(errors))
        else: 
            print('input is a legal density matrix')
    return legal
        
def generate_prob_lst(num_states):
    prob_lst = np.array([np.random.random() for _ in range(num_states)])
    prob_lst /= np.sum(prob_lst)
    return prob_lst

def get_rank(dm, tol=1e-10):
    return int(np.sum(np.linalg.eigvalsh(dm) > tol))

def get_fidelity(dm1, dm2, tol=1e-5):
    # assert is_legal(dm1) and is_legal(dm2), 'inputs are not legal density matrices'
    if not is_legal(dm1) and is_legal(dm2):
        print("Warning: inputs are not legal density matrices")
    try: 
        fidelity = (np.trace(sqrtm(sqrtm(dm1) @ dm2 @ sqrtm(dm1)))) ** 2
    except ValueError:
        print('fidelity cannot be computed for given inputs')
    # assert np.abs(np.imag(fidelity)) < tol, 'fidelity is not real within tol'
    return fidelity.real

def generate_dm(num_qubits, num_states, state_lst=None, prob_lst=None, prime_prob=None, compute_fidelity=False):
    assert (prob_lst is None) or (prime_prob is None), 'cannot set prob_lst and prime_prob together'
    if state_lst is None:
        state_lst = [random_statevector(2**num_qubits) for _ in range(num_states)]
        if compute_fidelity:
            prime_state = DensityMatrix(state_lst[0]).data
    if prime_prob is not None:
        prob_lst = np.array([prime_prob] + (generate_prob_lst(num_states - 1) * (1 - prime_prob)).tolist())
    elif prob_lst is None:
        prob_lst = generate_prob_lst(num_states)
    density_matrix = sum([DensityMatrix(state_lst[i]).data * prob_lst[i] for i in range(num_states)])
    if compute_fidelity:
        fidelity = get_fidelity(prime_state, density_matrix)
        return density_matrix, fidelity
    else:
        return density_matrix

def generate_random_01_strings(num_strings, length):
    characters = ['0', '1']
    generated_strings = []
    assert num_strings <= 2 ** length, 'too much strings to generate'
    for _ in range(num_strings):
        while True:
            random_string = ''.join(np.random.choice(characters) for _ in range(length))
            if random_string != '0' * length and random_string not in generated_strings:
                generated_strings.append(random_string)
                break
    return generated_strings

def generate_all_binary(length):
    all_strings = itertools.product('01', repeat=length)
    result = [''.join(s) for s in all_strings if '1' in s]
    return result

def generate_Pauli_expectations(dm, obsv):
    return np.trace(dm @ Pauli(obsv).to_matrix()).real

def get_trace_norm(dm):
    return np.sum(np.linalg.svd(dm, compute_uv=False))

In [ ]:
def generate_random_Pauli_strings(num_strings, num_qubits, pattern='balanced'):
    assert pattern in ['balanced', 'pro_I', 'pro_XYZ', 'uv_pair'], 'please choose pattern from: balanced, pro_I, pro_XYZ, uv_pair'
    generated_strings = []
    characters = ['X', 'Y', 'Z', 'I']
    assert 0 < num_strings <= 4 ** num_qubits - 1, 'too much or too few strings to generate'
    if pattern == 'balanced':
        for _ in range(num_strings):
            while True:
                random_string = ''.join(np.random.choice(characters) for _ in range(num_qubits))
                if random_string != 'I' * num_qubits and random_string not in generated_strings:
                    generated_strings.append(random_string)
                    break
        return generated_strings
    if pattern == 'uv_pair':
        uv_map = {'00':'I', '01':'Z', '10':'X', '11':'Y'}
        whole, remain = num_strings // 2 ** (num_qubits), num_strings % 2 ** (num_qubits)
        if whole > 0:
            v_lst = [format(i, f'0{num_qubits}b') for i in range(2 ** num_qubits)]
            u_lst = np.random.choice([format(i, f'0{num_qubits}b') for i in range(1, 2 ** num_qubits)], whole, replace=False).tolist()
            for u, v in list(itertools.product(u_lst, v_lst)):
                generated_strings.append(''.join([uv_map[u_char + v_char] for u_char, v_char in zip(u, v)]))
        if remain > 0:
            u_lst = ['0' * num_qubits]
            v_lst = np.random.choice([format(i, f'0{num_qubits}b') for i in range(1, 2 ** num_qubits)], remain, replace=False).tolist()
            for u, v in list(itertools.product(u_lst, v_lst)):
                generated_strings.append(''.join([uv_map[u_char + v_char] for u_char, v_char in zip(u, v)]))
        return generated_strings
    all_strings = generate_random_Pauli_strings(4 ** num_qubits - 1, num_qubits, pattern='balanced')
    grouped = dict()
    for i in range(num_qubits):
        grouped[i] = []
    for string in all_strings:
        grouped[string.count('I')].append(string)
    if pattern == 'pro_I':
        i = num_qubits - 1
        while len(generated_strings) < num_strings:
            if num_strings - len(generated_strings) >= len(grouped[i]):
                generated_strings += grouped[i]
            else:
                generated_strings += np.random.choice(grouped[i], num_strings - len(generated_strings), replace=True).tolist()
            i -= 1
        return generated_strings
    if pattern == 'pro_XYZ':
        i = 0
        while len(generated_strings) < num_strings:
            if num_strings - len(generated_strings) >= len(grouped[i]):
                generated_strings += grouped[i]
            else:
                generated_strings += np.random.choice(grouped[i], num_strings - len(generated_strings), replace=True).tolist()
            i += 1
        return generated_strings
        

In [ ]:
rho = generate_dm(3, 8)
all_Pauli = generate_random_Pauli_strings(63, 3)
all_expectations = [generate_Pauli_expectations(rho, obsv) for obsv in all_Pauli]
estm_rho = sum([Pauli(obsv).to_matrix() * expct for obsv, expct in zip(all_Pauli, all_expectations)]) / 2 ** 3 + Pauli('III').to_matrix() / 2 ** 3

In [ ]:
get_fidelity(rho, estm_rho)

In [ ]:
single_states = {'0': np.array([[1], [0]]), '1': np.array([[0], [1]])}

class State():
    def __init__(self, strings):
        if len(strings) == 1:
            self.state = single_states[strings]
        else:
            singles = [single_states[s] for s in strings]
            self.state = tensor_prod(*singles)
    def to_vector(self):
        return self.state
    
Hadamard = np.array([[1, 1], [1, -1]]) / np.sqrt(2)

S = np.array([[1, 0], [0, 1j]], dtype=np.complex128)

In [ ]:
# try to find the expectation of 'ZIZ'
print(f"the true expectation of ZIZ is {np.trace(rho @ Pauli('ZIZ').to_matrix()).real}")
all_states = [''.join(state) for state in list(itertools.product('01', '01', '01'))]
all_probs = [(State(state).to_vector().conj().T @ rho @ State(state).to_vector())[0][0].real for state in all_states]
p = sum(prob * (-1 if state[0] == '1' else 1) * (-1 if state[2] == '1' else 1) for prob, state in zip(all_probs, all_states))
print(f"the computed expectation of ZIZ is {p.real}")


In [ ]:
# try to find the expectation of 'XIZ'
print(f"the true expectation of XIZ is {np.trace(rho @ Pauli('XIZ').to_matrix()).real}")
all_states = [''.join(state) for state in list(itertools.product('01', '01', '01'))]
transformed_rho = tensor_prod(Hadamard, np.eye(4)).conj().T @ rho @ tensor_prod(Hadamard, np.eye(4))
all_transformed_probs = [(State(state).to_vector().conj().T @ transformed_rho @ State(state).to_vector())[0][0].real for state in all_states]
p = sum(prob * (-1 if state[0] == '1' else 1) * (-1 if state[2] == '1' else 1) for prob, state in zip(all_transformed_probs, all_states))
print(f"the computed expectation of ZIZ is {p.real}")


In [ ]:
# try to find the expectation of 'ZIY'
print(f"the true expectation of ZIY is {np.trace(rho @ Pauli('ZIY').to_matrix()).real}")
all_states = [''.join(state) for state in list(itertools.product('01', '01', '01'))]
transformed_rho = tensor_prod(np.eye(4), S @ Hadamard).conj().T @ rho @ tensor_prod(np.eye(4), S @ Hadamard)
all_transformed_probs = [(State(state).to_vector().conj().T @ transformed_rho @ State(state).to_vector())[0][0].real for state in all_states]
p = sum(prob * (-1 if state[0] == '1' else 1) * (-1 if state[2] == '1' else 1) for prob, state in zip(all_transformed_probs, all_states))
print(f"the computed expectation of ZIY is {p.real}")


In [ ]:
[''.join(observable) for observable in list(itertools.product(*['IXYZ' for _ in range(3)]))]